Импортируем библиотеки, используемые в дальнейшем

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Прочитаем значения из файла

In [2]:
filepath = "data/HW1_var_5.csv"
data_df = pd.read_csv(filepath, sep=';')
print("data_df shape:", data_df.shape)
data_df.head()

data_df shape: (10243, 44)


,Номер варианта,ID,INCOME_BASE_TYPE,CREDIT_PURPOSE,INSURANCE_FLAG,DTI,SEX,FULL_AGE_CHILD_NUMBER,DEPENDANT_NUMBER,EDUCATION,...,own_closed,min_MnthAfterLoan,max_MnthAfterLoan,dlq_exist,thirty_in_a_year,sixty_in_a_year,ninety_in_a_year,thirty_vintage,sixty_vintage,ninety_vintage
0,5,1000005,Поступление зарплаты на счет,Покупка недвижимости/ строительство,1,0.26,мужской,0,0,среднее,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,1000025,Поступление зарплаты на счет,Покупка автомобиля,1,0.45,мужской,1,0,высшее,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,1000045,2НДФЛ,Ремонт,1,0.49,мужской,0,0,высшее,...,0.0,66.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,1000065,Форма банка (без печати работодателя),Ремонт,1,0.38,мужской,1,0,Высшее/Второе высшее/Ученая степень,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1000085,Поступление зарплаты на счет,Ремонт,0,0.16,мужской,1,0,высшее,...,0.0,49.0,86.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Видим, что у нас имеется 44 колонки и 10243 записей

Далее выпишем значения по шаблону:

```
ключ - количество уникальных значений:
  уникальные значения
```

In [8]:
for key in data_df:
    nunique = data_df[key].nunique()
    print(f"{key} - {nunique}:")
    unique = data_df[key].unique()
    print("  ", unique)

Номер варианта - 1:
   [5]
ID - 10243:
   [1000005 1000025 1000045 ... 1204805 1204825 1204845]
INCOME_BASE_TYPE - 4:
   ['Поступление зарплаты на счет' '2НДФЛ'
 'Форма банка (без печати работодателя)'
 'Свободная форма с печатью работодателя' nan]
CREDIT_PURPOSE - 10:
   ['Покупка недвижимости/ строительство' 'Покупка автомобиля' 'Ремонт'
 'Покупка земли' 'Отпуск' 'Обучение' 'Покупка мебели'
 'Покупка бытовой техники' 'Другое' 'Лечение']
INSURANCE_FLAG - 2:
   [1 0]
DTI - 61:
   [0.26 0.45 0.49 0.38 0.16 0.2  0.41 0.19 0.31 0.43 0.18 0.22 0.47 0.53
 0.21 0.44 0.59 0.35 0.23  nan 0.32 0.55 0.24 0.27 0.37 0.34 0.52 0.56
 0.42 0.29 0.4  0.39 0.36 0.33 0.5  0.57 0.13 0.1  0.48 0.54 0.51 0.28
 0.3  0.46 0.25 0.17 0.58 0.12 0.14 0.08 0.15 0.11 0.03 0.09 0.06 0.01
 0.07 0.04 0.05 0.02 0.6  0.  ]
SEX - 2:
   ['мужской' 'женский']
FULL_AGE_CHILD_NUMBER - 7:
   [ 0  1  3  2  5  4 14]
DEPENDANT_NUMBER - 3:
   [0 2 1]
EDUCATION - 9:
   ['среднее' 'высшее' 'Высшее/Второе высшее/Ученая степень' 'вт